In [1]:
from jetcam.csi_camera import CSICamera
# Initialize the camera
camera = CSICamera(width=224, height=224)

In [2]:
camera.running = True
#hello

In [3]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import cv2
import numpy as np

In [4]:
# Create image preview widget
camera_widget = ipywidgets.Image(width=camera.width, height=camera.height)

# Link camera value to image widget
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Display the camera widget
display(camera_widget)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [20]:
# # Define the HSV range for the yellow color
# LOWER_YELLOW_HSV = np.array([11, 125, 50])
# UPPER_YELLOW_HSV = np.array([20, 255, 173])

# Define the HSV range for the yellow color
LOWER_YELLOW_HSV = np.array([11, 80, 87])
UPPER_YELLOW_HSV = np.array([22, 136, 182])

# Smoothing factor for center point calculation
ALPHA = 0.2

# Initialize previous center point
prev_center = None

In [25]:
def process_frame(frame):
    global prev_center

    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Threshold the HSV image to get only yellow colors
    yellow_mask = cv2.inRange(hsv, LOWER_YELLOW_HSV, UPPER_YELLOW_HSV)

    # Apply Gaussian blur to smooth the mask
    blurred = cv2.GaussianBlur(yellow_mask, (5, 5), 0)

    # Define the region of interest (ROI)
    height, width, _ = frame.shape
    roi_vertices = [
        (0, height),
        (0, height // 2),
        (width, height // 2),
        (width, height)
    ]
    mask = np.zeros_like(blurred)
    cv2.fillPoly(mask, np.array([roi_vertices], np.int32), 255)
    masked_blurred = cv2.bitwise_and(blurred, mask)

    # Edge detection using Canny
    edges = cv2.Canny(masked_blurred, 50, 150)

    # Hough Line Transform to detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=10, minLineLength=50, maxLineGap=20)

    # Create a blank image with the same dimensions as the original frame
    lane_image = np.copy(frame)

    # Initialize lists to store the coordinates of the leftmost and rightmost lines
    left_lines = []
    right_lines = []

    # Initialize center coordinates
    center_x = None
    center_y = None

    # Draw the lines on the blank image and store the leftmost and rightmost lines
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if y1 > height // 2 and y2 > height // 2:  # Filter lines in the ROI
                    if x1 < width // 2 and x2 < width // 2:
                        left_lines.append((x1, y1, x2, y2))
                    elif x1 > width // 2 and x2 > width // 2:
                        right_lines.append((x1, y1, x2, y2))

    # Draw the leftmost line
    if left_lines:
        left_line = sorted(left_lines, key=lambda line: line[0])[0]
        cv2.line(lane_image, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (0, 255, 0), 5)

    # Draw the rightmost line
    if right_lines:
        right_line = sorted(right_lines, key=lambda line: line[0])[-1]
        cv2.line(lane_image, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (0, 255, 0), 5)

    # Fill the area between the leftmost and rightmost lines
    if left_lines and right_lines:
        pts = np.array([
            [left_line[0], left_line[1]],
            [left_line[2], left_line[3]],
            [right_line[2], right_line[3]],
            [right_line[0], right_line[1]]
        ])
        cv2.fillPoly(lane_image, [pts], (0, 255, 0))

        # Calculate the center point between the two lines
        center_x = (left_line[0] + left_line[2] + right_line[0] + right_line[2]) // 4
        center_y = (left_line[1] + left_line[3] + right_line[1] + right_line[3]) // 4

        # Smoothing the center point
        if prev_center is not None:
            center_x = int(ALPHA * center_x + (1 - ALPHA) * prev_center[0])
            center_y = int(ALPHA * center_y + (1 - ALPHA) * prev_center[1])

        # Update previous center point
        prev_center = (center_x, center_y)

        # Ensure the center point is within the ROI
        if center_y > height // 2:
            # Draw a circle at the center point
            cv2.circle(lane_image, (center_x, height//2), 15, (0, 0, 255), -1)
        cv2.line(lane_image,(0,height//2), (width,height//2),(255,0,0),2)
        cv2.line(lane_image, (width // 2, 0), (width // 2, height), (255, 0, 0), 2)


    return lane_image, [center_x, center_y]

In [7]:
def map_value(value, from_low, from_high, to_low, to_high):
    # Calculate the mapped value
    if value == None:
        return None
    mapped_value = (value - from_low) * (to_high - to_low) / (from_high - from_low) + to_low
    return mapped_value

In [8]:
# This function will be called whenever the camera captures a new frame
def update_frame(change):
#     print("hello")
    frame = change['new']
    processed_frame,a = process_frame(frame)
#     b = map_value(a[0],0,224,-1,1)
#     print(b)
    camera_widget.value = bgr8_to_jpeg(processed_frame)

In [9]:
# Attach the update_frame function to the camera
camera.observe(update_frame, names='value')